# Plotting Titanic Prediction

This notebook is a tutorial of how we can modify the predictor python script to show a visualization of the data in the hidden space and how the predictor classifies each point. Each dimension of the hidden space corresponds to a neuron from the predictor. Because only two neurons are used for the titanic prediction model, we can plot it in the hidden space. 
*Note: This isn't part of Daimensions. We are going to manually edit the python script to output a png of the model's results in the hidden space.*

## 1. Build the Predictor

In [2]:
! btc -f NN titanic_train.csv -o vis_titanic.py -target Survived -rank -e 5 --yes


Brainome Daimensions(tm) 0.99 Copyright (c) 2019 - 2021 by Brainome, Inc. All Rights Reserved.
Licensed to:              Alexander Makhratchev  (Evaluation)
Expiration Date:          2021-04-30   56 days left
Number of Threads:        1
Maximum File Size:        30 GB
Maximum Instances:        unlimited
Maximum Attributes:       unlimited
Maximum Classes:          unlimited
Connected to:             daimensions.brainome.ai  (local execution)



Command:
    btc -f NN titanic_train.csv -o vis_titanic.py -target Survived -rank -e 5 --yes

Start Time:                 03/05/2021, 18:23

Attribute Ranking:
    Important columns:          Sex, SibSp, Parch, Pclass
    Overfit risk:                   0.0%
    Ignoring columns:           PassengerId, Name, Age, Ticket, Fare, Cabin, Embarked
    Test Accuracy Progression:     Sex :   78.68%
                                   SibSp :   79.69%  change   +1.01%
                                   Parch :   80.36%  change   +0.67%
                 

## 2. Make Changes to Python Script

The previous line of code should output a python script 'vis_titanic.py' with the model's predictor. Next, we'll manually edit the code by adding a few lines so that it will output the desired png when we run -validate. I'll show you the section of code we'll change with the modified lines highlighted.

In [3]:
# Helper (save an import)
def argmax(l):
    f = lambda i: l[i]
    return max(range(len(l)), key=f)
# Classifier
def single_classify(row, ax):
    #inits
    x = row
    o = [0] * num_output_logits


    #Nueron Equations
    h_0 = max((((-0.23418278 * float(x[0]))+ (0.2055268 * float(x[1]))+ (-3.683856 * float(x[2]))) + -0.5669335), 0)
    h_1 = max((((3.1762295 * float(x[0]))+ (0.29285425 * float(x[1]))+ (-0.0113988025 * float(x[2]))) + 1.4681039), 0)
    o[0] = (0.6990876 * h_0)+ (-0.89847755 * h_1) + 2.2160573

    #visualization modification
    ax.scatter(h_0, h_1, marker=('o' if o[0] >= 0 else '^'), c=('r' if o[0] >= 0 else 'b'))


    #Output Decision Rule
    if num_output_logits==1:
        return o[0]>=0
    else:
        return argmax(o)


def classify(arr):
    #init
    w_h = np.array([[-0.23418277502059937, 0.20552679896354675, -3.6838560104370117], [3.176229476928711, 0.2928542494773865, -0.011398802511394024]])
    b_h = np.array([-0.5669335126876831, 1.4681038856506348])
    w_o = np.array([[0.6990876197814941, -0.8984775543212891]])
    b_o = np.array(2.216057300567627)

    #Hidden Layer
    h = np.dot(arr, w_h.T) + b_h

    relu = np.maximum(h, np.zeros_like(h))


    #Output
    out = np.dot(relu, w_o.T) + b_o
    if num_output_logits == 1:
        return (out >= 0).astype('int').reshape(-1)
    else:
        return (np.argmax(out, axis=1)).reshape(-1)



def Predict(arr,headerless,csvfile, get_key, classmapping):
    
    with open(csvfile, 'r') as csvinput:
        #readers and writers
        writer = csv.writer(sys.stdout, lineterminator=os.linesep)
        reader = csv.reader(csvinput)

        #print original header
        if (not headerless):
            writer.writerow(','.join(next(reader, None) + ["Prediction"]))


        #visualization modification
        import matplotlib.pyplot as plt
        fig = plt.figure()
        ax = fig.add_subplot(111)

        for i, row in enumerate(reader):
            #use the transformed array as input to predictor
            pred = str(get_key(int(single_classify(arr[i],ax)), classmapping))
            #use original untransformed line to write out
            row.append(pred)
            writer.writerow(row)

        #visualization modification
        ax.set_xlabel('h0')
        ax.set_ylabel('h1')
        plt.savefig('titanic_visual.png')


## 3. Run -validate

You can make this modification in your own python script most easily by replacing lines 359-422 with the code above. Below I'm using an already modified version of the script called 'vis_titanic_mod.py', but you would use your edited 'vis_titanic.py'. We simply have to run the following line of code and it will output our desired png.

In [4]:
! python3 vis_titanic_mod.py titanic_test.csv

S,e,x,",",P,a,r,c,h,",",F,a,r,e,",",P,r,e,d,i,c,t,i,o,n
male,0,7.8292,0
female,0,7,1
male,0,9.6875,0
male,0,8.6625,0
female,1,12.2875,1
male,0,9.225,0
female,0,7.6292,1
male,1,29,0
female,0,7.2292,1
male,0,24.15,0
male,0,7.8958,0
male,0,26,0
female,0,82.2667,1
male,0,26,0
female,0,61.175,1
female,0,27.7208,1
male,0,12.35,0
male,0,7.225,0
female,0,7.925,1
female,0,7.225,1
male,0,59.4,0
male,1,3.1708,0
female,0,31.6833,1
male,1,61.3792,0
female,3,262.375,1
male,0,14.5,0
female,1,61.9792,1
male,0,7.225,0
male,0,30.5,0
male,0,21.6792,0
male,0,26,0
male,0,31.5,0
female,2,20.575,1
female,2,23.45,1
male,0,57.75,0
male,0,7.2292,0
female,0,8.05,1
female,0,8.6625,1
male,0,9.5,0
male,0,56.4958,0
male,1,13.4167,0
male,0,26.55,0
male,0,7.85,0
female,0,13,1
female,0,52.5542,1
male,0,7.925,0
male,0,29.7,0
male,0,7.75,0
female,0,76.2917,1
female,2,15.9,1
male,0,60,0
male,0,15.0333,0
female,1,23,1
female,2,263,1
male,0,15.5792,0
male,1,29.125,0
male,0,7.8958,0
male,0,7.65,0
male,0,16.1,0
female,0,262.3

Your current directory should now have a png called 'titanic_visual.png' in it that looks like the following: 
![titanic_visual](titanic_visual.png "Title")

Here we see the 2D hidden space that corresponds to the two neurons the model uses. We can see a clear plane separating the two classifications. This is the plane that the predictor uses to distinguish between the two classes.